<center><img src='https://drive.google.com/uc?id=1_utx_ZGclmCwNttSe40kYA6VHzNocdET' height="60">

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Program Operacyjny Polska Cyfrowa na lata 2014-2020
<hr>

<img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'>


Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej".   
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>

# Statistical machine learning - Notebook 6, version for students
**Author: Michał Ciach, Dorota Celińska-Kopczyńska**  


## Description


In today's class, we will learn how to perform linear regression.  
It's a method of fitting a linear function to a data set with a single predicted numerical variable $Y$ and several explanatory variables (a.k.a. predictors or independent variables) stored in a matrix $X$.  
The model of linear regression can be written as  
$$Y_i = X_i\beta + \epsilon_i,$$
where $X_i$ is the i-th row of the $X$ matrix and $\epsilon_i$ is the effect of factors influencing the value of $Y$ that are not included in $X$. This effect is called an *error*, but note that it's a slightly misleading name.   

When we want to include categorical variables as predictors (i.e. model the differences of income between voivodeships), we create so-called *dummy variables*: for each category of the variable, we create a binary dummy variable which is equal to 1 if a given observation comes from this category and 0 otherwise.

In [ ]:
!pip install gdown
!gdown https://drive.google.com/uc?id=1GW1pjKOCoKOlC4Jqbqql_ghYD_n0iC6O
!gdown https://drive.google.com/uc?id=1FInZ2jrlZGNColU4sHF9JKGHP39fTVut
!gdown https://drive.google.com/uc?id=1n1qS6dcVVKcVJOuUIIm0VTz6cSyrtzDH

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try 'pacman -S
    python-xyz', where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Arch-packaged Python package,
    create a virtual environment using 'python -m venv path/to/venv'.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip.
    
    If you wish to install a non-Arch packaged Python application,
    it may be easiest to use 'pipx install xyz', which will manage a
    virtual environment for you. Make sure you have python-pipx
    installed via pacman.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detailed specification.
zsh:1: no matches found: https://drive.google.com/uc?id=1GW1pjKOCoKOlC4Jqbqql_ghY

## Data & library imports

In this notebook, we'll introduce another Python library for statistical data analysis. The `statsmodels` library implements several statistical tests and methods for linear regression.

In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
from scipy.stats import norm, uniform
import statsmodels.api as sm

In [12]:
income = pd.read_csv('BDL municipality incomes 2015-2020.csv', sep=';', dtype={'Code': 'str'})
income.dropna(inplace=True)

**Excercise 1** (pen&paper or blackboard). Find OLS estimator for the model  $$y_i = \beta_0 + \beta_1x_{1i} + \beta_2x_{2i} + \varepsilon_i,$$ where $n = 4$,
$x_1^T = [1, 1, 2, −4]$, $x_2^T
= [−3, −3, 5, 1]$, $y^T = [1, 2, 3, 1]$. Find the residuals, the vector of the fitted values, and the value of $R^2$.

**Excercise 2** In this exercise, we'll learn how to estimate the linear regression in Python. Check your answers from Exercise 1 by constructing a linear regression model. Provide the interpretation for the coefficients.

In [93]:
import numpy as np
import statsmodels.api as sm

n = 4
x1 = np.array([1, 1, 2, -4])
x2 = np.array([-3, -3, 5, 1])
y = np.array([1, 2, 3, 1])

X = sm.add_constant(np.column_stack((x1, x2)))

print(X.shape, y.shape)

model = sm.OLS(y, X).fit()

fitted_values = model.fittedvalues
residuals = model.resid
r_squared = model.rsquared

print(model.summary())


(4, 3) (4,)
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.818
Model:                            OLS   Adj. R-squared:                  0.455
Method:                 Least Squares   F-statistic:                     2.250
Date:                Mon, 27 Nov 2023   Prob (F-statistic):              0.426
Time:                        15:29:19   Log-Likelihood:                -1.5169
No. Observations:                   4   AIC:                             9.034
Df Residuals:                       1   BIC:                             7.193
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.7500      0.354      4.

/usr/local/anaconda3/lib/python3.9/site-packages/statsmodels/stats/stattools.py:74: ValueWarning:

omni_normtest is not valid with less than 8 observations; 4 samples were given.



**Exercise 3 - Anscombe quartet** $R^2$ should help us in assessing the quality of our model. While a low value of this measure may encourage us to rework our model, unfortunately, a high value of $R^2$ is not enough to say we work with a good model. This exercise shows one of the weaknesses of this measure -- it may not be beneficial if we work with non-linear relationships.

Load the data from quartet.csv file. The file contains Anscombe's quartet -- a famous collection that comprises four datasets that have nearly identical simple descriptive statistics, yet have very different distributions and appear very different when graphed. Each dataset consists of eleven (x,y) points. They were constructed in 1973 by the statistician Francis Anscombe to demonstrate both the importance of graphing data when analyzing it, and the effect of outliers and other influential observations on statistical properties.

* Construct four linear models in the form $y_i \sim x_i$ where $i \in \{1,2,3,4\}$. Include intercepts!
* Compare and intepret the coefficients of determination of those models.
* Scatterplot $y_i$ and $x_i$  where $i \in \{1,2,3,4\}$ and add the fitted regression lines to the plots. Do you think those models are of high quality? Is the quality of the models the same?

In [94]:
anscombe = pd.read_csv('quartet.csv', sep=',')
anscombe

,x1,y1,x2,y2,x3,y3,x4,y4
0,10,8.04,10,9.14,10,7.46,8,6.58
1,8,6.95,8,8.14,8,6.77,8,5.76
2,13,7.58,13,8.74,13,12.74,8,7.71
3,9,8.81,9,8.77,9,7.11,8,8.84
4,11,8.33,11,9.26,11,7.81,8,8.47
5,14,9.96,14,8.10,14,8.84,8,7.04
6,6,7.24,6,6.13,6,6.08,8,5.25
7,4,4.26,4,3.10,4,5.39,19,12.50
8,12,10.84,12,9.13,12,8.15,8,5.56
9,7,4.82,7,7.26,7,6.42,8,7.91


In [95]:
models = []
x_values = []
y_values = []
for i in ['1', '2', '3', '4']:
    x = np.array([anscombe[f'x{i}']])
    y = np.array([anscombe[f'y{i}']])
#     print(x, y.shape)
    x_values.append(x)
    y_values.append(y)
    m = sm.OLS(y, x).fit()
    print(m.summary())
    models.append(m)

/usr/local/anaconda3/lib/python3.9/site-packages/statsmodels/stats/stattools.py:74: ValueWarning:

omni_normtest is not valid with less than 8 observations; 1 samples were given.



ValueError: shapes (1,11) and (1,11) not aligned: 11 (dim 1) != 1 (dim 0)

In [49]:
fig = px.scatter()

for i, (x, y, model) in enumerate(zip(x_values, y_values, models)):
    
    fig.add_trace(px.scatter(x=x, y=y).data[0])
    
    fig.add_hline(y=model)
    
fig.show()


ValueError: Cannot accept list of column references or list of columns for both `x` and `y`.

## Forecasting

**Exercise 4.** In this exercise, we'll learn how to use the linear regression for forecasting. Construct a linear regression model to explain the income of municipalities in 2017 (this is our dependent variable) based on the incomes from the years 2015 to 2016 (those are our independent variables).  
Calculate the RMSE (root mean squared error) of the fitted values.

Now, use this model to predict the incomes in 2018 based on the incomes from 2016 to 2017. Compute the RSS. Did the prediction error change? Why?

Predict the incomes in 2019 and 2020. Can you notice something particular in the RMSE values? What is the consequence for forecasting using machine learning models?

In [97]:
income = pd.read_csv('BDL municipality incomes 2015-2020.csv', sep=';', dtype={'Code': 'str'})
income.dropna(inplace=True)

income

,Code,Region,2015,2016,2017,2018,2019,2020
0,0201011,Bolesławiec (1),9.776646e+07,9.658595e+07,1.003549e+08,1.000265e+08,1.107985e+08,1.138563e+08
1,0201022,Bolesławiec (2),3.107224e+07,2.913815e+07,3.683091e+07,3.484836e+07,3.871533e+07,4.288890e+07
2,0201032,Gromadka (2),1.089941e+07,1.313974e+07,1.454154e+07,2.705794e+07,2.572157e+07,2.754443e+07
3,0201043,Nowogrodziec (3),1.856915e+07,2.941747e+07,3.188345e+07,3.178886e+07,3.913420e+07,3.341908e+07
4,0201052,Osiecznica (2),1.674647e+07,1.709802e+07,1.760182e+07,1.984173e+07,2.177626e+07,2.484304e+07
...,...,...,...,...,...,...,...,...
2504,3218043,Resko (3),1.523513e+07,1.448583e+07,1.611091e+07,1.763241e+07,1.760038e+07,1.941628e+07
2505,3218053,Węgorzyno (3),1.069708e+07,1.037428e+07,1.082646e+07,1.201310e+07,1.359405e+07,2.503944e+07
2506,3261011,Koszalin (1),2.697281e+08,2.891235e+08,3.054048e+08,3.245613e+08,3.432316e+08,3.840694e+08
2507,3262011,Szczecin (1),1.350327e+09,1.372046e+09,1.343422e+09,1.431826e+09,1.545381e+09,1.739014e+09


In [111]:
y_train = income['2017']
X_train = income[['2015', '2016']]

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [112]:
years = ['2015', '2016', '2017', '2018', '2019', '2020']
for i in range(4):
    predict = model.predict(income[[years[i], years[i+1]]])
    rmse = np.sqrt(mean_squared_error(income[years[i+2]], predict))
    print(rmse)


5116728.741660509
11957239.96221203
12234397.361899594
25873535.82886488


**Exercise 5 - homework.** In the cell below, you have a particular data set composed of the independent variable $X$ and four dependent variables $Y_1$ to $Y_4$. Create four linear regression models to predict $Y_i$ values based on $X$ using the `statsmodels` library.

Compare the estimated parameters of the model.  
Inspect the $R^2$ measures for the models.  
Which model fits best to its data?

Verify your conclusions by plotting the data sets on scatter plots and adding the fitted regression lines to them.

<center><img src='https://drive.google.com/uc?id=1_utx_ZGclmCwNttSe40kYA6VHzNocdET' height="60">

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Program Operacyjny Polska Cyfrowa na lata 2014-2020
<hr>

<img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'>


Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej".   
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>